In [2]:
import matplotlib.pyplot as plt
import os, re
import pandas as pd
import numpy as np

from datetime import datetime as dt
import cPickle as pickle
import re #Regex
import seaborn as sns
sns.set()
%matplotlib inline

In [3]:
DATAStd  = pd.DataFrame()
DATAPost = pd.DataFrame()

#Load standard striped Text
%time DATAStd['Judul'] = pickle.load( open( "v1.4\DATA_JUDULStriped-v4a.p", "rb" ) )
%time DATAStd['Short'] = pickle.load( open( "v1.4\DATA_SHORTStriped-v4a.p", "rb" ) )
%time DATAStd['Long']  = pickle.load( open( "v1.4\DATA_LONGStriped-v4a.p", "rb" ) )

#Load from PosTagged Noun and verb only Text
%time DATAPost['Judul'] = pickle.load( open( "v1.6\DATA_JUDULpostag-v5a.p", "rb" ) )
%time DATAPost['Short'] = pickle.load( open( "v1.6\DATA_SHORTpostag-v5a.p", "rb" ) )
%time DATAPost['Long']  = pickle.load( open( "v1.6\DATA_LONGpostag-v5a.p", "rb" ) )

#Load number of total tag in Text
%time AllFracJudul = pickle.load( open( "v2.0\DATA_JUDUL-v6a.p", "rb" ) )
%time AllFracShort = pickle.load( open( "v2.0\DATA_SHORT-v6a.p", "rb" ) )
%time AllFracLong  = pickle.load( open( "v2.0\DATA_LONG-v6a.p", "rb" ) )

#Load number of total noun-verb only in Text
# %time NVFracJudul = pickle.load( open( "v2.0\DATA_JUDUL-v2.p", "rb" ) )
# %time NVFracShort = pickle.load( open( "v2.0\DATA_SHORT-v2.p", "rb" ) )
# %time NVFracLong  = pickle.load( open( "v2.0\DATA_LONG-v2.p", "rb" ) )

%time DATA_TARGET = pickle.load( open( "v1.4\DATA_TARGET-v4a.p", "rb" ) )

DATA_TARGET = DATA_TARGET.reset_index(drop=True)
DATAStd  = DATAStd.reset_index(drop=True)
DATAPost = DATAPost.reset_index(drop=True)

Wall time: 181 ms
Wall time: 44 ms
Wall time: 265 ms
Wall time: 55 ms
Wall time: 36 ms
Wall time: 188 ms
Wall time: 164 ms
Wall time: 165 ms
Wall time: 159 ms
Wall time: 32 ms


In [4]:
#We only use 2% of total negative instance
factor = 100/100
size   = int(factor*DATA_TARGET[DATA_TARGET.values == -1].count())
print 'Total Negative Instances    : ',DATA_TARGET[DATA_TARGET.values == -1].count()
print 'Training Negative Instances : ',size
print 'Training Positif Instances  : ',DATA_TARGET[DATA_TARGET.values == 1].count()

Total Negative Instances    :  945
Training Negative Instances :  945
Training Positif Instances  :  399


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Stopwords data source : https://www.illc.uva.nl/Research/Publications/Reports/MoL-2003-02.text.pdf
stopword_html = open("id.stopwords.01.01.2016.txt",'r').read()
stopwords     = stopword_html.split("\n")

vect1 = TfidfVectorizer(decode_error='ignore')
vect2 = TfidfVectorizer(stop_words=stopwords,decode_error='ignore')

vectStds = vect1.fit_transform(DATAStd['Long'])
vectStop = vect2.fit_transform(DATAStd['Long'])
vectPost = vect1.fit_transform(DATAPost['Long'])

In [6]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

mnb = MultinomialNB(alpha=0.1)
#knn = KNeighborsClassifier()
svc = SVC(kernel='linear',cache_size=800,probability=True)
#tre = DecisionTreeClassifier(class_weight='balanced')

vectCnts = TfidfVectorizer(decode_error='ignore')
vectStds = TfidfVectorizer(decode_error='ignore')
#vectStop = TfidfVectorizer(stop_words=stopwords,decode_error='ignore')

kombinasi13 = [{'name' : 'MNB', 'pipe' : Pipeline([('vect', vectCnts),('clf', mnb)])},
               #{'name' : 'KNN', 'pipe' : Pipeline([('vect', vect),('clf', knn)])},
               {'name' : 'SVM', 'pipe' : Pipeline([('vect', vectCnts),('clf', svc)])},
               #{'name' : 'TREE', 'pipe' : Pipeline([('vect', vect),('clf', tre)])}
              ]

kombinasi24= [{'name' : 'MNB', 'pipe' : Pipeline([('vect', vectStds),('clf', mnb)])},
             #{'name' : 'KNN', 'pipe' : Pipeline([('vect', vect),('clf', knn)])},
             {'name' : 'SVM', 'pipe' : Pipeline([('vect', vectStds),('clf', svc)])},
             #{'name' : 'TREE', 'pipe' : Pipeline([('vect', vect),('clf', tre)])}
             ]

kombinasi5 = [{'name' : 'MNB', 'pipe' : mnb},
               {'name' : 'SVM', 'pipe' : svc},
             ]

In [7]:
k = 'aku akan memakan semua makanan itu'

def stopwordRemoval(text):
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

print stopwordRemoval(k)
JudulStop = DATAStd['Judul'].map(lambda x: stopwordRemoval(x))
ShortStop = DATAStd['Short'].map(lambda x: stopwordRemoval(x))
LongStop  = DATAStd['Long'].map(lambda x: stopwordRemoval(x))
Feed1Stop  = JudulStop+' '+ShortStop
Feed2Stop  = JudulStop+' '+LongStop+' '+ShortStop

JudulPostStop = DATAPost['Judul'].map(lambda x: stopwordRemoval(x))
ShortPostStop = DATAPost['Short'].map(lambda x: stopwordRemoval(x))
LongPostStop  = DATAPost['Long'].map(lambda x: stopwordRemoval(x))
Feed1PostStop  = JudulPostStop+' '+ShortPostStop
Feed2PostStop  = JudulPostStop+' '+LongPostStop+' '+ShortPostStop

print DATAStd['Judul'][198]
print JudulStop[198]

memakan makanan
hasil kunjungan menteri rini ke eropa, gandeng rolls royce
hasil kunjungan menteri rini eropa, gandeng rolls royce


In [8]:
X0 = []
# X0.append({'name':'Judul','Prep':'Kombinasi1','Data':DATAStd['Judul']})
# X0.append({'name':'Short','Prep':'Kombinasi1','Data':DATAStd['Short']})
# X0.append({'name':'Long' ,'Prep':'Kombinasi1','Data':DATAStd['Long']})
X0.append({'name':'X1' ,'Prep':'Kombinasi1','Data':DATAStd['Judul']+' '+DATAStd['Short']})
X0.append({'name':'X2' ,'Prep':'Kombinasi1','Data':DATAStd['Judul']+' '+DATAStd['Long']+' '+DATAStd['Short']})


# X1 = []
# X1.append({'name':'Judul','Prep':'Kombinasi2','Data':JudulStop})
# X1.append({'name':'Short','Prep':'Kombinasi2','Data':ShortStop})
# X1.append({'name':'Long' ,'Prep':'Kombinasi2','Data':LongStop})
# X1.append({'name':'Feed' ,'Prep':'Kombinasi2','Data':FeedStop})

# X2 = []
# X2.append({'name':'Judul','Prep':'Kombinasi3','Data':DATAPost['Judul']})
# X2.append({'name':'Short','Prep':'Kombinasi3','Data':DATAPost['Short']})
# X2.append({'name':'Long' ,'Prep':'Kombinasi3','Data':DATAPost['Long']})
# X2.append({'name':'Feed' ,'Prep':'Kombinasi3','Data':DATAPost['Judul']+' '+DATAPost['Short']})

X3 = []
# X3.append({'name':'Judul','Prep':'Kombinasi4','Data':JudulPostStop})
# X3.append({'name':'Short','Prep':'Kombinasi4','Data':ShortPostStop})
# X3.append({'name':'Long' ,'Prep':'Kombinasi4','Data':LongPostStop})
X3.append({'name':'X1' ,'Prep':'Kombinasi2','Data':Feed1PostStop})
X3.append({'name':'X2' ,'Prep':'Kombinasi2','Data':Feed2PostStop})

X4 = []
# X4.append({'name':'Judul','Prep':'Kombinasi5','Data':AllFracJudul})
# X4.append({'name':'Short','Prep':'Kombinasi5','Data':AllFracShort})
# X4.append({'name':'Long' ,'Prep':'Kombinasi5','Data':AllFracLong})
X4.append({'name':'X1' ,'Prep':'Kombinasi3','Data':AllFracJudul+AllFracShort})
X4.append({'name':'X2' ,'Prep':'Kombinasi3','Data':AllFracJudul+AllFracLong+AllFracShort})

# X2.append({'name':'Judul','Prep':'NVTag','Data':NVFracJudul})
# X2.append({'name':'Short','Prep':'NVTag','Data':NVFracShort})
# X2.append({'name':'Long' ,'Prep':'NVTag','Data':NVFracLong})

y_  = DATA_TARGET

In [9]:
from sklearn import grid_search
mnb_params = {'clf__alpha':[i*0.1 for i in range(10)]}

In [12]:
k13  = grid_search.GridSearchCV(kombinasi13[0]['pipe'],  mnb_params, cv=10,n_jobs=-1, scoring='roc_auc')
k24 = grid_search.GridSearchCV(kombinasi24[0]['pipe'], mnb_params, cv=10,n_jobs=-1, scoring='roc_auc')
k5  = grid_search.GridSearchCV(kombinasi5[0]['pipe'],  mnb_params, cv=10,n_jobs=-1, scoring='roc_auc')

In [32]:
kombinasi1[0]['pipe']

Pipeline(steps=[('tfidf', TfidfVectorizer(analyzer=u'word', binary=False, decode_error='ignore',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [17]:
for xi in X0:
    X_ = xi['Data']
    xP = X_[y_[y_.values == 1].index]
    xN = X_[y_[y_.values == -1].index][:size]
    X  = np.append(xP,xN)

    yP = y_[y_.values == 1]
    yN = y_[y_.values == -1][:size]
    y  = np.append(yP,yN)

    k13.fit(X,y)
    best_parameters = k13.best_estimator_.get_params()
    print best_parameters['clf__alpha']

0.2
0.1


In [18]:
for xi in X3:
    X_ = xi['Data']
    xP = X_[y_[y_.values == 1].index]
    xN = X_[y_[y_.values == -1].index][:size]
    X  = np.append(xP,xN)

    yP = y_[y_.values == 1]
    yN = y_[y_.values == -1][:size]
    y  = np.append(yP,yN)

    k24.fit(X,y)
    best_parameters = k24.best_estimator_.get_params()
    print best_parameters['clf__alpha']

0.2
0.1


In [20]:
for xi in X3:
    X_ = xi['Data']
    xP = X_[y_[y_.values == 1].index]
    xN = X_[y_[y_.values == -1].index][:size]
    X  = np.append(xP,xN)

    yP = y_[y_.values == 1]
    yN = y_[y_.values == -1][:size]
    y  = np.append(yP,yN)

    k13.fit(X,y)
    best_parameters = k13.best_estimator_.get_params()
    print best_parameters['clf__alpha']

0.2
0.1


In [ ]:
y2_  = pickle.load( open( "v2.0\DATA_TARGET-v5b.p", "rb" ) )
for xi in X5:
    X_ = xi['Data']
    xP = X_[y2_[y2_.values == 1].index]
    xN = X_[y2_[y2_.values == -1].index][:size]
    X  = np.vstack((xP,xN))

    yP = y2_[y2_.values == 1]
    yN = y2_[y2_.values == -1][:size]
    y  = np.append(yP,yN)

    k5.fit(X,y)
    best_parameters = k5.best_estimator_.get_params()
    print best_parameters['clf__alpha'],k4.best_score_